# 网页爬虫

不管是商业数据集，还是财经数据集，很多被嵌入到网页的结构和样式中，难以复制，需要使用特殊方法抓取出来。从网页中抽取数据的过程被称为**网络爬虫**。

## 1. 基本概念

客户端(Clients)通常是浏览器（Chrome、Edge、Safari），它们可以是任何类型的程序或设备。  
服务器(Servers)最常见的是云端服务器。
![clients_server](image/clients_server.png)

客户端(Clinet)和服务器(Server)之间的通信是通过请求(request)和响应(response)完成的：

1. 客户端,通常是浏览器向网络发送一个HTTP请求(request)
2. 服务器收到该请求
3. 服务器运行一个应用程序来处理该请求
4. 服务器向浏览器返回一个HTTP响应（response）
5. 浏览器收到该响应


以下是一些和网络爬虫相关，你需要知道的基本概念：

|名词|英文全称|概念解释|
|---|---|---|
|HTTP|Hypertext Transfer Protocol|超文本传输协议（HTTP）是互联网协议套件模型中的一个应用层协议，用于分布式、协作式、超媒体信息系统|
|URL|Uniform Resource Locator|它是互联网资源的地址，它包括一个协议，一个域名，有时还包括其他定位信息|
|认证|Authentication|它对访问者的身份进行验证，例如验证用户输入的密码是否正确|
|重定向|URL Redirections|它是一种网络技术，使一个网页在多个URL地址下可用|
|Cookies|HTTP Cookies|它是用户浏览网站时在本地存储的相关信息，包括身份信息、登录信息、购物车和历史记录等|


## 2. 使用urllib库抓取网页

urllib作为Python的标准库，基本上涵盖了基础的网络请求功能。

urllib中，request这个模块主要负责构造和发起网络请求，并在其中加入Headers、Proxy等。它还提供了一个稍微复杂的接口来处理常见的情况--如基本认证、cookies、代理等等。

使用 urllib.request 的最简单方法如下：

In [76]:
from urllib import request
resp = request.urlopen('http://www.baidu.com')
content = resp.read().decode('utf-8')   #根据网站对于文字的编码类型，进行解码

输出content的部分，例如前1000个字符：

In [77]:
content[:1000]

'<!DOCTYPE html><!--STATUS OK--><html><head><meta http-equiv="Content-Type" content="text/html;charset=utf-8"><meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1"><meta content="always" name="referrer"><meta name="theme-color" content="#ffffff"><meta name="description" content="全球领先的中文搜索引擎、致力于让网民更便捷地获取信息，找到所求。百度超过千亿的中文网页数据库，可以瞬间找到相关的搜索结果。"><link rel="shortcut icon" href="/favicon.ico" type="image/x-icon" /><link rel="search" type="application/opensearchdescription+xml" href="/content-search.xml" title="百度搜索" /><link rel="icon" sizes="any" mask href="//www.baidu.com/img/baidu_85beaf5496f291521eb75ba38eacbd87.svg"><link rel="dns-prefetch" href="//dss0.bdstatic.com"/><link rel="dns-prefetch" href="//dss1.bdstatic.com"/><link rel="dns-prefetch" href="//ss1.bdstatic.com"/><link rel="dns-prefetch" href="//sp0.baidu.com"/><link rel="dns-prefetch" href="//sp1.baidu.com"/><link rel="dns-prefetch" href="//sp2.baidu.com"/><link rel="apple-touch-icon-precomposed" href="https://psstatic.cd

上述返回的内容是一个使用HTML语言编写的字符串。HTML是超文本标记语言的缩写，它是网页的标准标记语言。上述内容包含了由<>标签表示的大量HTML元素。

## 3. 使用requests库抓取网页

[requests](https://docs.python-requests.org/en/latest/)是一个流行的爬虫工具，它在urllib3上进行封装，使得爬虫变得更加简单。

东方财富网是一家为用户提供基于互联网的财经资讯、数据等服务，旗下拥有财经门户“东方财富网”、互联网基金销售平台“天天基金”、中国人气股票基金交流社区“股吧”等。
为了方便下一章节文本分析需要使用的数据，我们访问股票社区网站，例如股吧：[http://guba.eastmoney.com/list,002594.html](http://guba.eastmoney.com/list,002594.html)。打开网页后，通过在firefox或者chrome里鼠标右键选择【检查元素】，就会看到以下图片：

![requests](image/requests.png)

**第一步，使用requests.get()函数获取html文件**

In [1]:
import requests
headers = { "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:91.0) Gecko/20100101 Firefox/91.0"}
url = 'http://guba.eastmoney.com/list,002594.html'
req = requests.get(url=url, headers=headers)
html_doc = req.content.decode("utf-8")

**第二步，我们解析图片里面这句话**

上图的`“完了！重大利空，锂电、新能源汽车明天又要”`在html的XPath路径为`“/html/body/div[6]/div[2]/div[4]/div[11]/span[3]/a”`。  
lxml是处理XML和HTML的Python库。使用方法见[官网](https://lxml.de/tutorial.html)。

In [11]:
from lxml import etree
html_obj = etree.HTML(html_doc)
reads_path = '/html/body/div[6]/div[2]/div[4]/div[11]/span[3]/a'
html_obj.xpath(reads_path)[0].text

'完了！重大利空，锂电、新能源汽车明天又要'

**第三步，我们设计一个函数，用来解析html文件里面的元素**

提取每一条评论的信息，包含阅读数、评论、标题、作者、最后更新时间等。

In [27]:
from lxml import etree
def parse_html(url, html_doc):
    """
    解析html获取数据
    """
    print("parse_html ...")
    html_obj = etree.HTML(html_doc)
    article_dict = {}
    i= 2 
    while True:
        try:
            reads_path = '/html/body/div[6]/div[2]/div[4]/div[%s]/span[1]' %i
            reads = html_obj.xpath(reads_path)[0].text

            comments_path = '/html/body/div[6]/div[2]/div[4]/div[%s]/span[2]' %i
            comments = html_obj.xpath(comments_path)[0].text

            post_path = '/html/body/div[6]/div[2]/div[4]/div[%s]/span[3]/a' %i
            post_href = html_obj.xpath(post_path)[0].get("href")
            post = html_obj.xpath(post_path)[0].get("title")

            writer_path = '/html/body/div[6]/div[2]/div[4]/div[%s]/span[4]/a/font' %i
            writer = html_obj.xpath(writer_path)[0].text

            update_path = '/html/body/div[6]/div[2]/div[4]/div[%s]/span[5]' %i
            update = html_obj.xpath(update_path)[0].text

            article_dict[i-1] = {"reads":reads,
                            "comments":comments, 
                            "post":post, 
                            "writer":writer, 
                            "update":update}
            i += 1
        except IndexError:
            break
    return article_dict

In [28]:
p = parse_html(url, html_doc)

parse_html ...


In [29]:
import pandas as pd
pd.DataFrame(p).T

,reads,comments,post,writer,update
1,3072,17,点评报告：3月销量重回20万辆，年度重点关注海外+智能化+高端化,比亚迪资讯,04-10 11:23
2,3071,32,国海证券给予比亚迪买入评级,比亚迪资讯,04-10 11:19
3,4053,18,行业首个！比亚迪拟4月10日发布云辇新能源智能车身控制系统,比亚迪资讯,04-10 11:19
4,5525,55,比亚迪云辇系统将于4月10日发布,比亚迪资讯,04-10 11:19
5,5198,18,攻入燃油车价格腹地：比亚迪加推冠军版车型应对“价格战”,比亚迪资讯,04-10 11:15
...,...,...,...,...,...
83,1363,32,巴菲特还是牛提前跑了。,文艺小青蛙,04-10 10:30
84,1310,41,利好,秃头方丈,04-10 10:29
85,495,1,又开始一路朝下了。。。,股友2i107g1539,04-10 10:28
86,2393,18,中石油2009年，巴菲特减持的，11元多，到现在，经过了10多年了，怎么样，中国,n528613702636584,04-10 10:28


### 练习

使用lxml.etree模块和其解析xpath()方法，访问股吧网页：[http://guba.eastmoney.com/list,002594.html](http://guba.eastmoney.com/list,002594.html)，抓取相关评论标题。

## 3.正则表达式的识别

正则表达式(Regular Expression)，又称规则表达式，是构成或分解字符串的规则。它使用单个字符串来描述、匹配一系列符合某个句法规则的字符串。

最常用的方法是`re.search`，其扫描整个字符串并返回第一个成功的匹配。

In [16]:
import re
pattern = 'www'
string = 'www.baidu.com'
print(re.search(pattern, string).span())               

(0, 3)


正则表达式模式如下：

![re_rules](image/re_rules.png)

### 练习

给定如下字符串，匹配出字符串'123'。

In [44]:
string = "abc123def"

## 4. 爬虫软件

这里推荐使用八爪鱼采集器，它是模拟人的采集方式，比如打开网页、点击网页的按钮等对网页上的信息进行全自动提取，提供了一种无需编程就可以爬取数据的方法。

安装方法如下：
1. 访问网址：https://www.bazhuayu.com
2. 安装完成后，注册账号。
3. 查看八爪鱼教程：https://www.bazhuayu.com/tutorial8/81zdypz

### 练习

使用八爪鱼软件爬取如下网站的帖子：http://guba.eastmoney.com/list,zssh000001.html ，并保存为CSV文件格式。

## 参考
- HTTP的概念: https://www.w3schools.com/whatis/whatis_http.asp
- Cookies: https://www.kaspersky.com/resource-center/definitions/cookies
- urllib官方文档: https://docs.python.org/3/library/urllib.request.html#  
- HOWTO Fetch Internet Resources Using The urllib Package:https://docs.python.org/3/howto/urllib2.html#urllib-howto